In [ ]:
# next generation of sensor code where the sensors run in their own 
# loops independently and the websockets dip in to the values


In [180]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [181]:
%sendtofile --quiet --source ../essential_sensor_code/ds3231.py
%sendtofile --quiet --source ../essential_sensor_code/ms5611.py


Sent 41 lines (2142 bytes) to ds3231.py.
Sent 75 lines (2561 bytes) to ms5611.py.


In [182]:
%sendtofile --quiet --source ../basicsockets/syslog.py

%sendtofile --quiet --source ../wificodes.txt
%sendtofile --quiet --source ../basicsockets/connectwifi.py
%sendtofile --quiet --source ../basicsockets/connecthotspot.py



Sent 34 lines (705 bytes) to syslog.py.
Sent 4 lines (87 bytes) to wificodes.txt.
Sent 34 lines (1042 bytes) to connectwifi.py.
Sent 9 lines (327 bytes) to connecthotspot.py.


In [183]:
%mpy-cross --set-exe /home/julian/extrepositories/micropython/mpy-cross/mpy-cross

%mpy-cross ../async_web_serve/webserve_funcs.py
%mpy-cross ../async_web_serve/websocket_funcs.py

%sendtofile --quiet --binary --source ../async_web_serve/webserve_funcs.mpy
%sendtofile --quiet --binary --source ../async_web_serve/websocket_funcs.mpy


Executing:  /home/julian/extrepositories/micropython/mpy-cross/mpy-cross ../async_web_serve/webserve_funcs.py
Executing:  /home/julian/extrepositories/micropython/mpy-cross/mpy-cross ../async_web_serve/websocket_funcs.py
Sent 4475 bytes in 150 chunks to webserve_funcs.mpy.
Sent 2427 bytes in 81 chunks to websocket_funcs.mpy.


In [184]:
%comment alternative commissioning with mpy files

%mpy-cross --set-exe /home/julian/extrepositories/micropython/mpy-cross/mpy-cross

%mpy-cross ../async_web_serve/uasyncio/__init__.py
%mpy-cross ../async_web_serve/uasyncio/core.py
%mpy-cross ../async_web_serve/uasyncio/synchro.py

%sendtofile --mkdir --binary --quiet --source ../async_web_serve/uasyncio/__init__.mpy lib/uasyncio/
%sendtofile --quiet --binary --source ../async_web_serve/uasyncio/core.mpy             lib/uasyncio/
%sendtofile --quiet --binary --source ../async_web_serve/uasyncio/synchro.mpy          lib/uasyncio/


Executing:  /home/julian/extrepositories/micropython/mpy-cross/mpy-cross ../async_web_serve/uasyncio/__init__.py
Executing:  /home/julian/extrepositories/micropython/mpy-cross/mpy-cross ../async_web_serve/uasyncio/core.py
Executing:  /home/julian/extrepositories/micropython/mpy-cross/mpy-cross ../async_web_serve/uasyncio/synchro.py
Sent 6317 bytes in 211 chunks to lib/uasyncio/__init__.mpy.
Sent 6352 bytes in 212 chunks to lib/uasyncio/core.mpy.
Sent 772 bytes in 26 chunks to lib/uasyncio/synchro.mpy.


In [185]:
%sendtofile --quiet --source ../sensor_websocket_graph/gcanvtest.html


Sent 214 lines (6238 bytes) to gcanvtest.html.


In [186]:
%sendtofile --source ../timeseries_analysis/lsq_funcs.py

Sent 45 lines (1416 bytes) to lsq_funcs.py.


In [194]:
%sendtofile --source main.py

Sent 219 lines (6624 bytes) to main.py.


In [ ]:
# use picocom -b115200 /dev/ttyUSB0 to connectand watch work when running from main

In [378]:
%disconnect


Closing serial Serial<id=0x7fac245de4a8, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [164]:
import os
for l in os.listdir():
    if l[-3:] == "dat":
        print(l, os.stat(l)[6])
        os.remove(l)
os.remove("syslog.txt")

In [382]:
%sendtofile --source main.py

Sent 178 lines (5591 bytes) to main.py.


In [375]:
%disconnect


Closing serial Serial<id=0x7fac210814a8, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [146]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [147]:
from connectwifi import ipnumber


LOG 9 ON 8392
Scanning wifis: DoESLiverpool, DoESDinky, DoESBackup, BV6000
LOG 12 Strongest wifi DoESBackup
> auth (b0)
> assoc (0)
> run (10)
..LOG 22 Device has ipnumber 192.168.0.162


In [187]:
import time
time.sleep(1.0)

from syslog import log, elog, powid
import os, ustruct, uasyncio

from webserve_funcs import servestaticfile
from webserve_funcs import servefilelist
from webserve_funcs import readhttpheaders

from webserve_funcs import recpostsave
from webserve_funcs import servemessage
from webserve_funcs import convertsavepostfile

from websocket_funcs import make_hybi07_header
from websocket_funcs import servewebsocket

from ms5611 import GetMS5611calibrations, MS5611convert, ams5611read
from ds3231 import rtctojsepoch, jsepochtoisodate

import machine, array 
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))

greenpin = machine.PWM(machine.Pin(13), freq=1024, duty=40)
touchpin = machine.TouchPad(machine.Pin(33))

jsepochoffset = rtctojsepoch(i2c, busywaitsec=True) - time.ticks_ms()
log(jsepochtoisodate(jsepochoffset))

# enable the wifi only if entry is held down on startup
ipnumber = ""

time.sleep_ms(100)
from connectwifi import ipnumber


LOG 51 ON 0
LOG 52 2017-11-28T20:01:38.854 
W (68062) phy_init: failed to load RF calibration data (0x1102), falling back to full calibration
Scanning wifis: DoESLiverpool, DoESDinky, DoESBackup, BV6000
LOG 55 Strongest wifi DoESLiverpool
.> auth (b0)
> assoc (0)
> run (10)

.LOG 63 Device has ipnumber 192.168.0.162


In [ ]:
# skip this if forcing result when opening from pc
for i in range(10):
    if touchpin.read() > 1100:
        break
    greenpin.duty(500 if ((i%2) == 0) else 0)
    time.sleep_ms(200)
    touchpin.read()
else:
    # this should be deferred or made on-off
    for i in range(20):
        greenpin.duty(500 if ((i%2) == 0) else 0)
        time.sleep_ms(50)
    from connectwifi import ipnumber
    #from connecthotspot import ipnumber
    for i in range(20):
        greenpin.duty(500 if ((i%2) == 0) else 0)
        time.sleep_ms(50)


In [188]:
import array, ustruct
from lsq_funcs import setpt0, addpt, copylq, mergelq, calcmc, calcrsq, dn

lqR = array.array("f", [0]*7)
lqA = array.array("f", [0]*7)
lqS = array.array("f", [0]*7)
nbatch = 32
rlimsq = 2.2**2

def nextpt(t, v, fout):
    if lqR[dn] == 0:
        setpt0(lqR, t, v)
        return
    addpt(lqR, t, v)
    if lqR[dn] < nbatch:
        return
    if lqA[dn] == 0:
        copylq(lqA, lqR)
        lqR[dn] = 0
        return
    copylq(lqS, lqA)
    mergelq(lqS, lqR)
    m, c = calcmc(lqS)
    rsq = calcrsq(lqS, m, c)
    if rsq > rlimsq:
        print("rsq", lqA[dn], rsq)
        if fout:
            fout.write(ustruct.pack("<fffffff", *lqA))
            fout.flush()
        copylq(lqA, lqR)
    else:
        copylq(lqA, lqS)
    lqR[dn] = 0


In [189]:
hook_ms5611 = [1, 0, 0, None]

async def ams5611():
    log("Entering ams5611")
    bdat = open("b%d_%d.dat"%(powid, time.ticks_ms()//1000), "wb")
    nbdat = 0
    try:
        dc = GetMS5611calibrations(i2c)
        readout = bytearray(3)
        while hook_ms5611[0]:
            await ams5611read(dc, readout, i2c, uasyncio)
            t = time.ticks_ms()
            hook_ms5611[2] = MS5611convert(dc)
            if bdat:
                nextpt(t, hook_ms5611[2], bdat)
                nbdat += 1
                if nbdat > 10000:
                    bdat.close()
                    bdat = None
            if hook_ms5611[3]:  # dump out to logfile here (if open)
                hook_ms5611[3].write(ustruct.pack("<II", hook_ms5611[1], hook_ms5611[2]))
                if (hook_ms5611[1]>>13) != (t>>13):   # 8 seconds
                    hook_ms5611[3].flush()
            hook_ms5611[1] = t
            greenpin.duty((t%500) if hook_ms5611[3] or (((t//500)%3) == 0) else 0)
    except Exception as e:
        elog(e)
    hook_ms5611[0] = -1
    log("Leaving ams5611")

# waits and looks to detect the touchpin touched
async def createLogFile():
    ti = 0
    while hook_ms5611:
        ti = (ti + 1  if touchpin.read()<1100  else 0)
        if ti > 3:
            log("making datfile")
            hook_ms5611[3] = open("%d_%d.dat"%(powid, time.ticks_ms()//1000), "wb")
            for i in range(12*10):    # 10 minutes 
                await uasyncio.sleep_ms(5*1000)   # 5 second chunks
                if touchpin.read()<1100:
                    break   # break early
            hook_ms5611[3].close()
            hook_ms5611[3] = None
        await uasyncio.sleep_ms(50)
    


In [190]:
# coroutine for handling the write loop (and closing when note comes through thr reader)
async def wswritesensorloop(rpath, writer, wscondition):
    rpath = rpath.split(",")
    
    delayms = int(rpath[1])
    hexbuff = bytearray(16)
    #if rpath[0] == "bme280":
    try:
        while wscondition[1]:
            while len(wscondition) > 2:
                incoming = wscondition.pop(2)
                log("[WS%d incoming] %s"%(wscondition[0], incoming))
                
            await writer.awrite(make_hybi07_header(16))
            ustruct.pack_into(">II", hexbuff, 8, hook_ms5611[1], hook_ms5611[2])
            for i in range(8):
                b = hexbuff[i+8]
                hexbuff[i*2] = (b>>4) + (48 if b < 160 else 55)  #ord('0')=48; ord('A')=65
                b &= 15
                hexbuff[i*2+1] = b + (48 if b < 10 else 55)
            await writer.awrite(hexbuff)
            await uasyncio.sleep_ms(delayms)
            
    except OSError as e:
        elog(e)
        wscondition[1] = 1
    log("[WS%d wpleave]"%wscondition[0])



In [191]:
async def handleconnection(reader, writer):
    receivedrequest = await readhttpheaders(reader)
    log(receivedrequest)
    
    if "POST" in receivedrequest:   # this is causing the str and binary comparison warning
        await recpostsave(receivedrequest, reader, "POSTUPLOAD.txt")
        await servemessage(writer, "done")
        await writer.aclose()
        #await reader.aclose()  # this gets a keyerror
        convertsavepostfile("POSTUPLOAD.txt")
        return
    
    rpath = receivedrequest.get("GET")
    
    bcallaclose = True
    if "WebSocketKey" in receivedrequest:
        bcallaclose = await servewebsocket(rpath, reader, writer, receivedrequest["WebSocketKey"], wswritesensorloop)
    elif rpath is None:
        pass
    elif rpath == "":
        await servefilelist(writer)
    else:
        await servestaticfile(writer, rpath)
        
    if bcallaclose:
        await writer.aclose()


import gc
async def garbagecollect():
    while True:
        gc.collect()
        await uasyncio.sleep_ms(20000)



In [192]:

# enables rewriting handleconnection function without restarting
def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))
    

port = 80
loop = uasyncio.get_event_loop()
loop.create_task(garbagecollect())
if ipnumber:
    print("Will listen on http://%s:%s/" % (ipnumber, port))
    loop.create_task(uasyncio.start_server(handleconnection_indirect, ipnumber, port))
loop.create_task(ams5611())
loop.create_task(createLogFile())




Will listen on http://192.168.0.162:80/


In [34]:
#loop.create_task(abme280(i2c, hook_bme280))
#loop.create_task(uasyncio.start_server(handleconnection_indirect, ipnumber, port))


In [193]:
#%sendtofile --append main.py

loop.run_forever()


LOG 82 Entering ams5611 
LOG 85 {'GET': ''} 
.LOG 89 {} 
LOG 90 {'GET': ''} 
.LOG 95 {'GET': ''} 
.rsq 320.0 9.76129e+07
LOG 100 {'GET': ''} 
rsq 32.0 1.342177e+08
.rsq 96.0 1.342177e+08
.rsq 160.0 8.947848e+07
.rsq 96.0 1.342177e+08
.rsq 64.0 8.947848e+07
...rsq 320.0 9.76129e+07
rsq 64.0 8.947848e+07
...............LOG 211 {'GET': 'gcanvtest.html'} 
LOG 211 File 'gcanvtest.html' (6239 bytes) served in 272ms 
LOG 212 {'WebSocketKey': b'piPRYtggaOcI97bK+RsLOw==', 'GET': 'bme280,50'} 
LOG 212 [WS0] b'Hello'
LOG 212 [WS0 incoming] b'Hello' 
LOG 216 [WS0 close] 
LOG 216 [WS0 rlleave] 
LOG 216 [WS0 wpleave] 
.LOG 219 {'GET': ''} 
.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
  File "../async_web_serve/uasyncio/core.py", line 65, in run_forever
  File "../async_web_serve/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


In [353]:
%sendtofile --source gcanvtest.html

Sent 214 lines (6237 bytes) to gcanvtest.html.


In [301]:
help(gc)
print(gc.mem_alloc(), gc.mem_free())

object <module 'gc'> is of type module
  __name__ -- gc
  collect -- <function>
  disable -- <function>
  enable -- <function>
  isenabled -- <function>
  mem_free -- <function>
  mem_alloc -- <function>
  threshold -- <function>
55504 40496


In [176]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
OSError: [Errno 2] ENOENT
MicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [178]:
import os
#print(os.remove("static/graphtest.html"))
print(os.listdir("static"))


['\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x00\x00\x00\x00.\x00\x00\x00', '\x00\x00\x00\x00\x

In [322]:
help(gc)

object <module 'gc'> is of type module
  __name__ -- gc
  collect -- <function>
  disable -- <function>
  enable -- <function>
  isenabled -- <function>
  mem_free -- <function>
  mem_alloc -- <function>
  threshold -- <function>


In [188]:
ai = socket.getaddrinfo(ipnumber, 80)
print(ai)


[(2, 1, 0, '192.168.43.6', ('192.168.43.6', 80))]


In [189]:
addr = ai[0][4]
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(addr)


In [192]:
gc.collect()

In [193]:
s.listen(10)


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: [Errno 12] ENOMEM


In [143]:
import os
os.remove("webserve_funcs.py")
os.remove("websocket_funcs.py")

In [144]:
%mpy-cross ../async_web_serve/webserve_funcs.py
%mpy-cross ../async_web_serve/websocket_funcs.py
%sendtofile --quiet --binary --source ../async_web_serve/webserve_funcs.mpy
%sendtofile --quiet --binary --source ../async_web_serve/websocket_funcs.mpy


Executing:  /home/julian/extrepositories/micropython/mpy-cross/mpy-cross ../async_web_serve/webserve_funcs.py
Executing:  /home/julian/extrepositories/micropython/mpy-cross/mpy-cross ../async_web_serve/websocket_funcs.py
Sent 4475 bytes in 150 chunks to webserve_funcs.mpy.
Sent 2427 bytes in 81 chunks to websocket_funcs.mpy.


In [219]:
%disconnect


Closing serial Serial<id=0x7fac210814a8, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
